# TASK 1: User Overview analysis

In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
sys.path.append('../scripts')
from Clean_data import clean_data

In [3]:
# Import the dataset
df = pd.read_csv("../data/Week1_challenge_data_source(CSV).csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

## Sub-tasks

>## Identifying the top 10 handsets used by the customers </br>

The handsets type of the customers are stored in the column named `Handset Type`. But before extracting the information for this column, we should identify each unique user/customer. As a matter of fact, a customer can be identified by either his/her IMSI, MSISDN/number or IMEI.

In [4]:
# Search the column that have the less number of missing values
IdVariable = clean_data(df[['IMSI','MSISDN/Number', 'IMEI','Handset Type']])
missingCol,_,_ = IdVariable.missing_values(verbose=False)
missingCol

MSISDN/Number    1066
IMEI              572
Handset Type      572
IMSI              570
dtype: int64

Thus one can choose to use the `IMSI` to identify each customer because we get more information using this field instead of `IMEI` or `MSISDN/Number`. Nethertheless, one can also choose to use the `IMEI`. Since we're looking for the handset type, we can just focus on the **IMEI** because it's an unique number which identify a device on a mobile network. Furthermore, without the IMEI, we can't identify the handset type - this is the reason why we get the same number of missing values for those two columns. Even if, we get less missing values for IMSI, this caracteristics does not ensure that we'll get the maximum of information relative to the handset type.

In [5]:
# Extract the unique customer from the table
UniqueUser = df.loc[:,['IMEI','Handset Manufacturer','Handset Type']].dropna(how="all")
# Drop the duplicates
UniqueUser = UniqueUser.drop_duplicates()
# Count the number of each handset type and identify the top 10
UniqueUser.loc[:,'Handset Type'].value_counts()[:10]

Huawei B528S-23A                10638
Apple iPhone 6S (A1688)          6765
undefined                        6716
Apple iPhone 6 (A1586)           6271
Apple iPhone 7 (A1778)           4721
Apple iPhone Se (A1723)          3764
Apple iPhone 8 (A1905)           3550
Samsung Galaxy S8 (Sm-G950F)     3275
Apple iPhone Xr (A2105)          3077
Samsung Galaxy J5 (Sm-J530)      2760
Name: Handset Type, dtype: int64

In [6]:
# Count the number of each handset type and identify the top 10 (without undefined handset type)
UniqueUser.query("`Handset Type`!='undefined'").loc[:,'Handset Type'].value_counts()[:10]

Huawei B528S-23A                10638
Apple iPhone 6S (A1688)          6765
Apple iPhone 6 (A1586)           6271
Apple iPhone 7 (A1778)           4721
Apple iPhone Se (A1723)          3764
Apple iPhone 8 (A1905)           3550
Samsung Galaxy S8 (Sm-G950F)     3275
Apple iPhone Xr (A2105)          3077
Samsung Galaxy J5 (Sm-J530)      2760
Samsung Galaxy A5 Sm-A520F       2721
Name: Handset Type, dtype: int64

>## Identify the three top handset manufacturer

In [7]:
def topNManufacturer(df,topn=3):
    toNMan = df.loc[:,'Handset Manufacturer'].value_counts()[:topn]
    return toNMan

In [8]:
# Identify the top 3 manufacturer
topNManufacturer(UniqueUser,topn=3)

Apple      42687
Samsung    30981
Huawei     21743
Name: Handset Manufacturer, dtype: int64

>## Identify the top 5 handsets per top 3 handset manufacturer

In [9]:
# Function to extract the top n type of handset for the top m manufacturer
def topTypeManufact(df=UniqueUser,nmanufact=3,ntype=5):
    topNManufact = df.loc[:,'Handset Manufacturer'].value_counts()[:nmanufact]
    res = pd.DataFrame(columns=['Manufacturer','Type','Count'])
    for manufacturer in topNManufact.index:
        temp = df.loc[UniqueUser['Handset Manufacturer']==manufacturer,'Handset Type'].value_counts()[:ntype]
        temp = pd.DataFrame({'Manufacturer':[manufacturer]*ntype,'Type':temp.index,'Count':temp.to_list()})
        res = pd.concat([res,temp])
    return res.reset_index(drop=True)

In [10]:
# Identify the top 5 handsets per top 3 handset manufacturer
topTypeManufact(df=UniqueUser,nmanufact=3,ntype=5)

,Manufacturer,Type,Count
0,Apple,Apple iPhone 6S (A1688),6765
1,Apple,Apple iPhone 6 (A1586),6271
2,Apple,Apple iPhone 7 (A1778),4721
3,Apple,Apple iPhone Se (A1723),3764
4,Apple,Apple iPhone 8 (A1905),3550
5,Samsung,Samsung Galaxy S8 (Sm-G950F),3275
6,Samsung,Samsung Galaxy J5 (Sm-J530),2760
7,Samsung,Samsung Galaxy A5 Sm-A520F,2721
8,Samsung,Samsung Galaxy J3 (Sm-J330),2606
9,Samsung,Samsung Galaxy S7 (Sm-G930X),2310


>## Task 1.1: Get an overview of the users’ behavior on those applications

In [15]:
df[['IMSI','Bearer Id']].groupby("IMSI").count()

,Bearer Id
IMSI,
2.040471e+14,1
2.040808e+14,1
2.082001e+14,1
2.082001e+14,1
2.082001e+14,2
...,...
2.082099e+14,1
2.082099e+14,1
2.082099e+14,1


In [17]:
df[['IMSI','Dur. (ms)']].groupby("IMSI").sum()

,Dur. (ms)
IMSI,
2.040471e+14,869844.0
2.040808e+14,877385.0
2.082001e+14,86019.0
2.082001e+14,86254.0
2.082001e+14,573923.0
...,...
2.082099e+14,127367.0
2.082099e+14,168543.0
2.082099e+14,86399.0


In [23]:
df[['IMSI','Total UL (Bytes)','Total DL (Bytes)']].groupby("IMSI").sum()

,Total UL (Bytes),Total DL (Bytes)
IMSI,,
2.040471e+14,60456049.0,78697597.0
2.040808e+14,37295915.0,194828056.0
2.082001e+14,53794658.0,31376981.0
2.082001e+14,33583272.0,392918313.0
2.082001e+14,71325004.0,961767023.0
...,...,...
2.082099e+14,33888182.0,688815403.0
2.082099e+14,44692550.0,451173021.0
2.082099e+14,30355487.0,307825104.0


In [22]:
df[['IMSI','Total UL (Bytes)','Total DL (Bytes)']].groupby("IMSI").sum().sum(axis=1)

IMSI
2.040471e+14    1.391536e+08
2.040808e+14    2.321240e+08
2.082001e+14    8.517164e+07
2.082001e+14    4.265016e+08
2.082001e+14    1.033092e+09
                    ...     
2.082099e+14    7.227036e+08
2.082099e+14    4.958656e+08
2.082099e+14    3.381806e+08
2.082522e+14    8.457132e+08
2.140743e+14    6.945057e+08
Length: 107265, dtype: float64

In [34]:
dfData = df.loc[:,['IMSI','Social Media DL (Bytes)','Social Media UL (Bytes)',
                                  'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
                                  'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
                                  'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
                                  'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)',
                                  'Total UL (Bytes)', 'Total DL (Bytes)']]

In [45]:
SocialMediaCol = [col for col in dfData.columns if 'Social Media' in col]
print(SocialMediaCol)

['Social Media DL (Bytes)', 'Social Media UL (Bytes)']


In [49]:
dfData[SocialMediaCol].sum(1)

0         1.570185e+06
1         1.933278e+06
2         1.726277e+06
3         6.574930e+05
4         9.127880e+05
              ...     
149996    3.517065e+06
149997    2.352181e+06
149998    1.260239e+06
149999    8.231090e+05
150000    1.828250e+06
Length: 150001, dtype: float64

In [33]:
df.iloc[:,39:55].columns.to_list()

['Social Media DL (Bytes)',
 'Social Media UL (Bytes)',
 'Google DL (Bytes)',
 'Google UL (Bytes)',
 'Email DL (Bytes)',
 'Email UL (Bytes)',
 'Youtube DL (Bytes)',
 'Youtube UL (Bytes)',
 'Netflix DL (Bytes)',
 'Netflix UL (Bytes)',
 'Gaming DL (Bytes)',
 'Gaming UL (Bytes)',
 'Other DL (Bytes)',
 'Other UL (Bytes)',
 'Total UL (Bytes)',
 'Total DL (Bytes)']

In [ ]:
df.dropna()

In [ ]:
np.sum()